In [ ]:
import lithops

bucketname = 'scipy-tutorial-txtfiles'

In [ ]:
# Map function definition
# This is the function that will be executed in parallel over different workers
def my_map_function(obj):
    print(f'Bucket: {obj.bucket}')
    print(f'Key: {obj.key}')
    print(f'Partition num: {obj.part}')
    counter = {}

    data = obj.data_stream.read()

    for line in data.splitlines():
        for word in line.decode('utf-8').split():
            if word not in counter:
                counter[word] = 1
            else:
                counter[word] += 1

    return counter

In [ ]:
# Reduce function definition
# This function collects all the map results and aggregate them in the final one
def my_reduce_function(results):
    final_result = {}
    for count in results:
        for word in count:
            if word not in final_result:
                final_result[word] = count[word]
            else:
                final_result[word] += count[word]

    return final_result

In [ ]:
# Executor declaration and job call in a very simple way
fexec = lithops.FunctionExecutor()
fexec.map_reduce(my_map_function, bucketname, my_reduce_function)
print(fexec.get_result())